In [5]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.path.append('..')

In [7]:
import logging
from datetime import datetime, timezone
from typing import List

from redis.asyncio import Redis

from app.database_redis.connection import get_redis_client
from app.database_redis.dals.connection_dal import ConnectionDAL
from app.services.apis.streamqueue_service.client import StreamQueueServiceAPI
from app.services.apis.streamqueue_service.schemas import AudioChunkInfo, ExistingConnectionInfo
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings

from app.services.audio.audio import AudioSlicer


check_and_process_connections_interval_sec: 1.0


In [8]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

## copy meeting for tests

In [9]:
meeting_id = 'wcg-kmhp-snd'

In [17]:
meeting = Meeting(redis_client, meeting_id)
await meeting.load_from_redis()

In [18]:
meeting_copy = Meeting(redis_client,'test_meeting_id')

In [23]:
for t in meeting_copy.timestamps:
    v = getattr(meeting,t)
    setattr(meeting_copy,t,v)

In [24]:
meeting_copy.diarizer_seek_timestamp

datetime.datetime(2024, 6, 4, 11, 35, 57, 670813, tzinfo=tzutc())

In [25]:
await meeting_copy.update_redis()

In [13]:
connections = await meeting.get_connections()

In [14]:
len(connections)

16

In [15]:
[c.id for c in connections]

['7bb8e2dc-b195-4e9e-b2d1-c671b59163ac',
 'f859aa3a-24db-47f0-824e-cd57c26ed87b',
 '4aa7e520-b12c-41a9-9d61-d27c4aaf23c2',
 '735bdc55-b5a9-4229-8406-b2d22ade9fcd',
 '3824169c-d0f8-40a6-9aba-f8dfa8dd39dc',
 'e09c5679-225c-4be1-9be6-54db98a91b88',
 '7f9c37e0-0283-42dc-bb6c-d3b101812c1f',
 '2fd56fe0-7b29-4bb9-86eb-72c34a0e865c',
 '2304e753-4526-41a1-94bc-26db021a69cf',
 'f93488ff-7f17-4cbb-ab5a-6b7b1b694000',
 '6858450a-4add-48b3-8741-3cc99f1b3b3e',
 'fc990593-03e7-4bc0-9b0e-46459c292e8e',
 '209763a5-c4a4-4524-b371-865213418ee1',
 '1aaf6374-6e41-476e-aa5e-97543ba7e211',
 '50b159d8-15d4-415e-b312-275186b1e3c0',
 'd0614519-8718-414e-a95b-c21a98f68986']

In [17]:
[await meeting_copy.add_connection(c.id) for c in connections]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [20]:
connections = await meeting_copy.get_connections()

In [21]:
len(connections)

16

In [72]:
current_time = datetime.now(timezone.utc)

In [73]:
diarizer_step = 30

In [74]:
if (current_time - meeting.diarizer_last_updated_timestamp).seconds > diarizer_step:
    print("diarizer added")
    diarizer = Diarizer(redis_client)
    await diarizer.add_todo(meeting.meeting_id)
    await meeting.update_diarizer_timestamp(
        segment_start_timestamp, diarizer_last_updated_timestamp=current_time
    )

diarizer added


NameError: name 'segment_start_timestamp' is not defined